In [4]:
from rosemary import jpt_setup; jpt_setup()
import os; os.environ['CUDA_VISIBLE_DEVICES'] = '0'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:

import logging
import math
from pathlib import Path
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional
import json
import numpy as np
import pickle
from functools import partial

import pyarrow
import datasets
import evaluate
import torch
import torch.nn as nn
from datasets import load_dataset, IterableDataset

import numpy as np

import transformers
from transformers import (
    CONFIG_MAPPING,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    DataCollatorForLanguageModeling,
    is_torch_tpu_available,
    set_seed,
)
from transformers.testing_utils import CaptureLogger
from transformers.trainer_utils import get_last_checkpoint
from transformers.trainer_callback import TrainerState
from transformers.trainer import TRAINER_STATE_NAME
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from transformers.models.gpt2.configuration_gpt2 import GPT2Config

from doremi.training_args import ModelArguments, DataTrainingArguments, FullTrainingArguments
import doremi.dataloader as data_utils
from doremi.trainer import DoReMiTrainer
from doremi.dataloader import determine_skip_per_domain
from doremi.dataloader import interleave_datasets


try:
    import doremi.models as doremi_models
except Exception:
    
    pass
try:
    from flash_attn.models.gpt_neox import gpt_neox_config_to_gpt2_config
except Exception:
    pass


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.27.0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt")

logger = logging.getLogger(__name__)

In [6]:
package_dir = "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi"
cache_dir = '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache'
preprocessed_data = "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed"

envs = {
    "CACHE": cache_dir,
    "DOREMI_DIR": package_dir,
    "PILE_DIR": os.path.join(package_dir, "data", 'raw'),
    "PREPROCESSED_PILE_DIR": preprocessed_data,
    "MODEL_OUTPUT_DIR": os.path.join(package_dir, 'results'),
    "PARTITION": "el8",
    "HF_HOME": cache_dir,
    "TRANSFORMERS_CACHE": cache_dir,
    "HF_DATASETS_CACHE": cache_dir,
    "HF_DATASETS_IN_MEMORY_MAX_SIZE": "0",
    "TORCH_EXTENSIONS_DIR": cache_dir,
    "TMPDIR": cache_dir,
    "WANDB_DIR": os.path.join(cache_dir, "wandb"),
    "WANDB_MODE": 'offline',
    "PREPROCESSED_DATA": preprocessed_data,
    'PREPROCESSED_CACHE': os.path.join(cache_dir, 'preprocessed_cache', 'perdomain_pile_preprocessed'),

}

for k, v in envs.items():
    os.environ[k] = v
    
os.makedirs(cache_dir, exist_ok=True)


In [47]:
job_name = 'train_baseline'

num_gpus = 1
nodes = 1
use_doremi = False
doremi_optimizer = 'doremiv2'

hf_models_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/'
model_type = 'gpt2'
model_name_or_path = os.path.join(hf_models_dir, 'gpt2-medium'); abbr_model_name = 'gpt2-medium'
model_name_or_path = ('/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/doremi/results/'
                      'ft1/gpt2-medium_humanmix_doremi'); abbr_model_name = 'gpt2-medium'
model_name_or_path = os.path.join(hf_models_dir, 'gpt2-xl'); abbr_model_name = 'gpt2-xl'
model_name_or_path = os.path.join(hf_models_dir, 'EleutherAI/pythia-160m'); model_type = 'pythia'; abbr_model_name = 'pythia-160m'


cache_dir = envs['CACHE']
abbr_train_file = 'humanmix_uniform'
domain_config_path = os.path.abspath(f'../configs/{abbr_train_file}_baseline_50kvocab.json')
dataset_dir = preprocessed_data


total_batch_size = 128 # # 64*8=512
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
gradient_accumulation_steps = int(total_batch_size/(num_gpus*nodes)/per_device_train_batch_size)
max_steps = int(200000/total_batch_size); save_steps = 10 # 200k steps.
gradient_checkpointing = True

print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{per_device_train_batch_size} batch size per GPU, "
      f"{gradient_accumulation_steps} gradient accumulation steps,"
      f"for {max_steps} max steps.")


fsdp = "full_shard auto_wrap"
fsdp = False
if 'gpt2' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'GPT2Block'
elif 'llama' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'LlamaDecoderLayer'
elif 'mpt' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'MPTBlock'
elif 'pythia' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'GPTNeoXLayer'
else: raise ValueError('Not sure how to set `fsdp_transformer_layer_cls_to_wrap`')
    

# use `dataset_dir` instead of `dataset_name` to specify `preprocessed_dir`
# --dataset_name=pile \

## learning rate for pretraining, substituted with finetuning hyperparameters
# --learning_rate 1e-3 \
# --lr_end 1e-4 \
# --adam_epsilon 1e-8 \

## don't need cosine scheduling for finetuning
# --weight_decay 0.01 \
# --lr_scheduler_name linear_warmup_cosine \
# --warmup_ratio 0.06 \

## avoids grad scaling error
# --fp16 \
## for training model from scratch
# --config_overrides="n_positions=1024,n_embd=1024,n_layer=18,n_head=16" \

## added the following
# add_domain_id: for non-pile preprocessed dataset
# do_padding: true for variable size sequences, as in instruction tuning datasets.
# --max_train_samples 1000 \

output_dirname = f'{abbr_model_name}_{abbr_train_file}'
if use_doremi:
    output_dirname += '_doremi'
else:
    output_dirname += '_baseline'
output_dirname = 'jpt_'+output_dirname
output_dir = os.path.join(envs['MODEL_OUTPUT_DIR'], output_dirname)


if use_doremi:
    reference_model_name_or_path = (
        '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/doremi/results/'
        'ft1/gpt2-medium_humanmix_baseline/')
    doremi_options = f"""
        --doremi_optimizer={doremi_optimizer} \
        --reweight_eta=1 \
        --reweight_eps=1e-4 \
        --train_domain_weights_tmp_file={os.path.join(output_dir, 'domain_weights')} \
        --reweight_domains \
        --remove_unused_columns=False \
        --reference_model_name_or_path={reference_model_name_or_path} \
    """
else:
    doremi_options = ''

cmd = f"""
--model_name_or_path={model_name_or_path} \
--model_type={model_type} \
--do_train \
--cache_dir={cache_dir} \
--dataset_dir={dataset_dir} \
--domain_config_path={domain_config_path} \
--max_token_length=1024 \
--per_device_train_batch_size={per_device_train_batch_size} \
--gradient_accumulation_steps={gradient_accumulation_steps} \
--dataloader_num_workers=1 \
--learning_rate=2e-5 \
--lr_scheduler_type=linear \
--warmup_ratio=0.03 \
--weight_decay=0. \
--max_grad_norm=1.0 \
--max_steps={max_steps} \
--evaluation_strategy=no \
--save_strategy=steps \
--save_steps={save_steps} \
--save_total_limit=1 \
--run_name={job_name} \
--seed=1111 \
--logging_strategy=steps \
--logging_steps=10 \
--logging_first_step \
--report_to='none' \
--optim=adamw_hf \
--adam_beta1=0.9 \
--adam_beta2=0.99 \
--gradient_checkpointing \
{'--fsdp_transformer_layer_cls_to_wrap="'+fsdp_transformer_layer_cls_to_wrap+'"' 
    if fsdp else ''} \
{'--gradient_checkpointing' if gradient_checkpointing  else ''} \
--add_domain_id=True \
--do_padding=True \
{doremi_options if doremi_options else ''} \
--overwrite_output_dir \
--output_dir={output_dir} \
"""
# --overwrite_output_dir \

import shlex
args = shlex.split(cmd)

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, FullTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)
model_args, data_args, training_args

[INFO|training_args.py:1301] 2023-08-08 22:38:52,920 >> Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
[INFO|training_args.py:1716] 2023-08-08 22:38:52,921 >> PyTorch: setting up devices


Training /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/EleutherAI/pythia-160m using 1 GPUs, 1 batch size per GPU, 128 gradient accumulation steps,for 1562 max steps.


(ModelArguments(model_name_or_path='/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/EleutherAI/pythia-160m', model_type='pythia', config_overrides=None, config_name=None, tokenizer_name=None, cache_dir='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache', use_fast_tokenizer=True, model_revision='main', use_auth_token=False, torch_dtype=None),
 DataTrainingArguments(dataset_dir='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed', dataset_name='', max_train_samples=None, max_eval_samples=None, max_token_length=1024, block_size=None, overwrite_cache=False, do_padding=True, add_domain_id=True, preprocessing_num_workers=None, shuffle=True),
 FullTrainingArguments(output_dir='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results/jpt_pythia-160m_humanmix_uniform_baseline', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=False, evaluation_strategy=<IntervalStrateg

In [50]:

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

if training_args.should_log:
    # The default of training_args.log_level is passive, so we set log level at info here to have that default.
    transformers.utils.logging.set_verbosity_info()
    

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()


# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

08/08/2023 22:39:20 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
08/08/2023 22:39:20 - INFO - __main__ - Training/evaluation parameters FullTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.99,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=1,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=False,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
domain_config_path=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/configs/humanmix_uniform_baseline_50kvocab.json,
doremi_optimizer=doremiv1,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1

In [18]:

# Detecting last checkpoint.
last_checkpoint = None
num_skip_examples = 0
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )
        state = TrainerState.load_from_json(str(Path(last_checkpoint) / TRAINER_STATE_NAME))
        global_batch_size = training_args.train_batch_size * training_args.gradient_accumulation_steps * training_args.world_size
        num_skip_examples = state.global_step * global_batch_size
        logger.info(f"Skipping {num_skip_examples} examples")
        
last_checkpoint, num_skip_examples

(None, 0)

In [19]:

# Set seed before initializing model.
set_seed(training_args.seed)

In [37]:

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

config_kwargs = {
    "cache_dir": model_args.cache_dir,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
    "use_cache": True if not training_args.gradient_checkpointing else False,
}
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, **config_kwargs)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, **config_kwargs)
    # wpq: `max_model_length` not set correctly if load pre-downloaded models on disk.
    if model_args.model_type == 'gpt_neox_flash':
        config = gpt_neox_config_to_gpt2_config(config)
        config.use_flash_attn = True
        config.fused_mlp = True
        config.fused_bias_fc = True
        config.fused_dropout_add_ln = True
        config.pad_vocab_size_multiple = 8
        config.activation_function = 'gelu_new'
        config.n_inner = None
        # disable absolute
        config.max_position_embeddings = 0
else:
    if model_args.model_type == 'gpt_flash': 
        config = GPT2Config(
                vocab_size=50257, n_positions=2048, n_embd=2048,
                n_layer=24, n_head=16, 
                scale_attn_by_inverse_layer_idx=True, 
                rotary_emb_fraction=0.5,
                use_flash_attn=True, fused_mlp=True,
                fused_bias_fc=True, fused_dropout_add_ln=True, 
                pad_vocab_size_multiple=8)
        # disable absolute
        config.max_position_embeddings = 0
    elif model_args.model_type == 'gpt_neox_flash':
        # convert to GPT2 config
        config = CONFIG_MAPPING['gpt_neox']() 
        config = gpt_neox_config_to_gpt2_config(config)
        config.use_flash_attn = True
        config.fused_mlp = True
        config.fused_bias_fc = True
        config.fused_dropout_add_ln = True
        config.pad_vocab_size_multiple = 8
        config.activation_function = 'gelu_new'
        config.n_inner = None
        # disable absolute
        config.max_position_embeddings = 0
    else:
        config = CONFIG_MAPPING[model_args.model_type]()
    # wpq: `max_model_length` not set correctly if load pre-downloaded models on disk.
    logger.warning("You are instantiating a new config instance from scratch.")
    if model_args.config_overrides is not None:
        logger.info(f"Overriding config: {model_args.config_overrides}")
        config.update_from_string(model_args.config_overrides)
        logger.info(f"New config: {config}")


[INFO|configuration_utils.py:710] 2023-08-08 22:34:04,796 >> loading configuration file /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/EleutherAI/pythia-160m/config.json
[INFO|configuration_utils.py:768] 2023-08-08 22:34:04,798 >> Model config GPTNeoXConfig {
  "_name_or_path": "/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/EleutherAI/pythia-160m",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float

In [38]:

# wpq: `max_model_length` not specified correctly when loading pre-downloaded models from disk.
# here specify `max_model_length` so that data collator can function properly.
if model_args.model_type == 'gpt2':
    model_max_length = 1024
elif model_args.model_type in {'llama', 'pythia'}:
    model_max_length = 2048
else:
    model_max_length = None

tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
    "model_max_length": model_max_length,
}

if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)

elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )
    
tokenizer

[INFO|tokenization_utils_base.py:1837] 2023-08-08 22:34:42,119 >> loading file vocab.json
[INFO|tokenization_utils_base.py:1837] 2023-08-08 22:34:42,120 >> loading file merges.txt
[INFO|tokenization_utils_base.py:1837] 2023-08-08 22:34:42,121 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:1837] 2023-08-08 22:34:42,122 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:1837] 2023-08-08 22:34:42,123 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:1837] 2023-08-08 22:34:42,124 >> loading file tokenizer_config.json


GPTNeoXTokenizerFast(name_or_path='/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/EleutherAI/pythia-160m', vocab_size=50254, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [39]:

if model_args.model_name_or_path:
    torch_dtype = (
        model_args.torch_dtype
        if model_args.torch_dtype in ["auto", None]
        else getattr(torch, model_args.torch_dtype)
    )
    if model_args.model_type in {'gpt_flash', 'gpt_neox_flash'}:
        model = doremi_models.GPTFlashAttnLMHeadModel.from_pretrained(
            model_args.model_name_or_path, config=config)
    else:
        if model_args.model_type in {'gpt2'}:
            model_cls = doremi_models.GPT2LMHeadModelDoReMi
        elif model_args.model_type in {'pythia'}:
            model_cls = doremi_models.GPTNeoXForCausalLMDoReMi
        else:
            model_cls = AutoModelForCausalLM
        model = model_cls.from_pretrained(
            model_args.model_name_or_path, 
            config=config,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
            torch_dtype=torch_dtype,
        )
else:
    if model_args.model_type in {'gpt_flash', 'gpt_neox_flash'}:
        model = doremi_models.GPTFlashAttnLMHeadModel(config)
    elif model_args.model_type in {'gpt2'}:
        model = doremi_models.GPT2LMHeadModelDoReMi(config)
    elif model_args.model_type in {'pythia'}:
        model = doremi_models.GPTNeoXForCausalLMDoReMi(config)
    else:
        model = AutoModelForCausalLM.from_config(config)

    n_params = sum({p.data_ptr(): p.numel() for p in model.parameters()}.values())
    logger.info(f"Training new model from scratch - Total size={n_params/2**20:.2f}M params")
    
model

[INFO|modeling_utils.py:2604] 2023-08-08 22:36:27,998 >> loading weights file /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/EleutherAI/pythia-160m/model.safetensors
[INFO|configuration_utils.py:603] 2023-08-08 22:36:28,017 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "transformers_version": "4.32.0.dev0",
  "use_cache": false
}

[INFO|modeling_utils.py:3333] 2023-08-08 22:36:30,456 >> All model checkpoint weights were used when initializing GPTNeoXForCausalLMDoReMi.

[INFO|modeling_utils.py:3341] 2023-08-08 22:36:30,457 >> All the weights of GPTNeoXForCausalLMDoReMi were initialized from the model checkpoint at /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/EleutherAI/pythia-160m.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPTNeoXForCausalLMDoReMi for predictions without

GPTNeoXForCausalLMDoReMi(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
    

In [ ]:
# from typing import Tuple, Union
# from transformers import GPT2LMHeadModel
# # from doremi.models import CausalLMOutputWithDomainIDs
# from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions
# from torch.nn import CrossEntropyLoss

# @dataclass
# class CausalLMOutputWithDomainIDs(CausalLMOutputWithCrossAttentions):
#     domain_ids: Optional[torch.LongTensor] = None
#     reference_pertoken_loss: Optional[torch.FloatTensor] = None  # corresponds to uniq_domain_ids
#     pertoken_loss: Optional[torch.FloatTensor] = None  # corresponds to uniq_domain_ids
#     token_mask: Optional[torch.BoolTensor] = None  # 1 for tokens that are not padding



# #             model_args.model_name_or_path,
# #             from_tf=bool(".ckpt" in model_args.model_name_or_path),
# #             config=config,
# #             cache_dir=model_args.cache_dir,
# #             revision=model_args.model_revision,
# #             use_auth_token=True if model_args.use_auth_token else None,
# #             torch_dtype=torch_dtype,

# # model = doremi_models.GPTFlashAttnLMHeadModel.from_pretrained(
# #     model_args.model_name_or_path, config=config)

        
# model = GPTLMHeadModelDoReMi.from_pretrained(
#     model_args.model_name_or_path, config=config)


In [ ]:
# collate_fn = data_utils.get_data_collator(tokenizer, do_padding=data_args.do_padding)

# batch = list(train_dataset.take(3))
# batch = collate_fn(batch)
# batch = {k: v.to('cpu') for k,v in batch.items()}
# print(batch['input_ids'].shape)
# print(model.device, batch['input_ids'].device)

# out = model(**batch, return_pertoken_losses=True, )
# print(out.pertoken_loss.shape, out.token_mask.shape)

In [40]:

with open(training_args.domain_config_path, 'r') as f:
    domain_config = json.load(f)

train_domain_weights_dict = domain_config['train_domain_weights']
eval_domain_weights_dict = domain_config['eval_domain_weights']
# whenever we convert dict to array, we sort by key
domain_list = list(sorted(train_domain_weights_dict.keys()))
num_domains = len(domain_list)

print(domain_list, num_domains, train_domain_weights_dict)

if training_args.do_train:
    # data script could change tokenizer shape
    train_dataset = data_utils.get_preprocessed_mixed_dataset(
            preprocessed_dir=data_args.dataset_dir,
            domain_weights_dict=train_domain_weights_dict,
            dataset_name=data_args.dataset_name,
            cache_dir=model_args.cache_dir,
            split='train',
            max_samples=data_args.max_train_samples,
            add_domain_id=data_args.add_domain_id,
            tmp_file=None,
            seed=training_args.seed,
            tokenizer=tokenizer,
            shuffle=data_args.shuffle,
            num_skip_examples=num_skip_examples,
            shard_reversal=training_args.reweight_domains,
            training_args=training_args,
            data_args=data_args,
    )

if training_args.do_eval:
    eval_dataset = data_utils.get_preprocessed_mixed_dataset(
            preprocessed_dir=data_args.dataset_dir,
            domain_weights_dict=eval_domain_weights_dict,
            dataset_name=data_args.dataset_name,
            cache_dir=model_args.cache_dir,
            split='validation',
            add_domain_id=data_args.add_domain_id,
            max_samples=data_args.max_eval_samples,
            tokenizer=tokenizer,
            no_interleave=True,
            training_args=training_args,
            data_args=data_args,
    )

['cot', 'dolly', 'flan_v2', 'oasst1'] 4 {'cot': 0.25, 'flan_v2': 0.25, 'dolly': 0.25, 'oasst1': 0.25}
08/08/2023 22:37:06 - WARNING - doremi.dataloader - No split used or split directory not found: using same data for all splits.
08/08/2023 22:37:06 - INFO - datasets.builder - Using custom data configuration default-20e8d8f5df875937
08/08/2023 22:37:06 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/json
08/08/2023 22:37:06 - INFO - datasets.builder - Overwrite dataset info from restored data version if exists.
08/08/2023 22:37:06 - INFO - datasets.info - Loading Dataset info from /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96
08/08/2023 22:37:06 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2

  0%|          | 0/1 [00:00<?, ?it/s]

08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-5e2824c3a7de4516_00000_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-5e2824c3a7de4516_00001_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-5e2824c3a7de4516_00002_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

  0%|          | 0/1 [00:00<?, ?it/s]

08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-839697b853eec600_00000_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-839697b853eec600_00001_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-839697b853eec600_00002_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

...

08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-84a80786f2af044e_00000_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-84a80786f2af044e_00001_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-84a80786f2af044e_00002_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

  0%|          | 0/1 [00:00<?, ?it/s]

08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-731e81448230ef08_00000_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-731e81448230ef08_00001_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-731e81448230ef08_00002_of_00008.arrow
08/08/2023 22:37:06 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

[ERROR|tokenization_utils_base.py:1056] 2023-08-08 22:37:06,740 >> Using pad_token, but it is not set yet.


08/08/2023 22:37:06 - INFO - datasets.builder - Using custom data configuration default-96635e9b3ed9d7c9
08/08/2023 22:37:06 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/generator
08/08/2023 22:37:06 - INFO - datasets.builder - Using custom data configuration default-deea1cceea3d2a7b
08/08/2023 22:37:06 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/generator
08/08/2023 22:37:06 - INFO - datasets.builder - Using custom data configuration default-c2c2291c2e13d44c
08/08/2023 22:37:06 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/generator
08/08/2023 22:37:06 - INFO - datasets.builder - Using custom data configuration default-c65a69

In [ ]:

# preprocessed_dir=data_args.dataset_dir
# domain_weights_dict=train_domain_weights_dict
# dataset_name=data_args.dataset_name
# cache_dir=model_args.cache_dir
# split='train'
# max_samples=data_args.max_train_samples
# add_domain_id=data_args.add_domain_id
# tmp_file=None
# seed=training_args.seed
# tokenizer=tokenizer
# shuffle=data_args.shuffle
# num_skip_examples=num_skip_examples
# shard_reversal=training_args.reweight_domains
# no_interleave=False

# print(preprocessed_dir)
# print(domain_weights_dict)
# print(dataset_name)
# print(cache_dir)
# print(split)
# print(max_samples)
# print(add_domain_id)
# print(seed)
# print(shuffle)
# print(num_skip_examples)
# print(shard_reversal)


# domain_names = list(sorted(domain_weights_dict.keys()))
# domain_to_idx = {domain_names[i]: i for i in range(len(domain_names))}
# domain_weights = np.asarray([domain_weights_dict[domain_name] for domain_name in domain_names])
# domain_weights = domain_weights / domain_weights.sum()

# print()
# print(json.dumps({'domain_names': domain_names, 
#                   'domain_to_idx': domain_to_idx, 
#                   'domain_weights': list(domain_weights)},
#                 indent=4))


# # write domain weights to file if tmp_file is set
# if tmp_file is not None:
#     probabilities_tmp_file = tmp_file

#     with open(str(probabilities_tmp_file), 'wb') as f:
#         pickle.dump(domain_weights, f)
#     probabilities = None
# else:
#     probabilities = domain_weights
#     probabilities_tmp_file = None


# print()
# print(json.dumps({'probabilities': list(probabilities)}, indent=4))

# # from doremi.dataloader import get_perdomain_datasets
# # all_ds = get_perdomain_datasets(
# #     preprocessed_dir, 
# #     domain_weights_dict,
# #     cache_dir=cache_dir,
# #     split=split,
# #     seed=seed,
# #     domain_weights=domain_weights,
# #     domain_names=domain_names,
# #     num_skip_examples=num_skip_examples,
# #     shuffle=shuffle,
# #     shard_reversal=shard_reversal
# # )

# domain_name_to_skip_num = determine_skip_per_domain(num_skip_examples, seed, domain_weights, domain_names)

# preprocessed_dir = Path(preprocessed_dir)
# if split is not None and (preprocessed_dir / split).exists():
#     preprocessed_dir = preprocessed_dir / split
# else:
#     logger.warn(f"No split used or split directory not found: using same data for all splits.")

# domains = list(sorted(domain_weights_dict.keys()))

# print(preprocessed_dir)
# print(domain_name_to_skip_num)
# print()
# print(json.dumps({'preprocessed_dir': str(preprocessed_dir), 
#                   'domain_name_to_skip_num': domain_name_to_skip_num}, indent=4))


# all_ds = {}
# for domain in domains:
#     domain_dir = preprocessed_dir / domain
    
#     ## wpq: read instruction tuning dataset off `jsonl` files
#     if (domain_dir / f'{domain}_data.jsonl').exists():
#         from datasets import load_dataset
#         from functools import partial
#         from open_instruct.finetune_trainer import encode_with_prompt_completion_format, encode_with_messages_format
#         from doremi.dataloader import skippable_data_gen_dataset

#         data_files = {'train': str(domain_dir / f'{domain}_data.jsonl')}
#         raw_datasets = load_dataset(
#             "json",
#             data_files=data_files,
#             cache_dir=cache_dir,
#             use_auth_token=True if model_args.use_auth_token else None,
#         )
#         # Preprocessing the datasets.
#         if "prompt" in raw_datasets["train"].column_names and "completion" in raw_datasets["train"].column_names:
#             encode_function = partial(
#                 encode_with_prompt_completion_format,
#                 tokenizer=tokenizer,
#                 max_seq_length=1024,
#             )
#         elif "messages" in raw_datasets["train"].column_names:
#             encode_function = partial(
#                 encode_with_messages_format,
#                 tokenizer=tokenizer,
#                 max_seq_length=1024,
#             )
#         else:
#             raise ValueError("You need to have either 'prompt'&'completion' or 'messages' in your column names.")

#         with training_args.main_process_first(local=False, desc="Processing instruction data"):
#             lm_datasets = raw_datasets.map(
#                 encode_function,
#                 num_proc=16,
#                 batched=False,
#             )
#             lm_datasets.set_format(type="pt")
#         ds = lm_datasets['train']
#         ds = IterableDataset.from_generator(
#                 skippable_data_gen_dataset,
#                 gen_kwargs={'ds': ds,
#                             'num_skip_examples': domain_name_to_skip_num[domain],
#                             'loop': (split == 'train'),
#                             'seed': seed,
#                             'shuffle': shuffle}
#                 )
#         seed += 1
#     elif (domain_dir / 'dataset_info.json').exists():
#         ds = load_from_disk(dataset_path=str(domain_dir))
#         logger.info(f"Loaded {domain_dir}. Length: {len(ds)}")
#     else:
#         curr_shards = list(domain_dir.iterdir())
#         if shard_reversal:
#             curr_shards = list(reversed(curr_shards))
#         # shuffle shard order
#         random.Random(seed).shuffle(curr_shards)
#         ds = IterableDataset.from_generator(
#                 skippable_data_gen,
#                 gen_kwargs={'shards': curr_shards,
#                             'num_skip_examples': domain_name_to_skip_num[domain],
#                             'loop': (split == 'train'),
#                             'seed': seed,
#                             'shuffle': shuffle}
#                 )
#         seed += 1
#     all_ds[domain] = ds
    

# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

# def add_domain_id_generator(ds, domain_idx):
#     for ex in ds:
#         ex['domain_id'] = domain_idx
#         yield ex
        
# domain_ds_ls = []
# for domain_name in domain_names:
#     domain_idx = domain_to_idx[domain_name]
#     domain_ds = all_ds[domain_name]
#     # add domain_id if necessary
#     if add_domain_id:
#         domain_ds = IterableDataset.from_generator(
#             add_domain_id_generator, 
#             gen_kwargs={'ds': domain_ds, 'domain_idx': domain_idx})
#     domain_ds_ls.append(domain_ds)

# if no_interleave:
#     # instead of interleaving, run through each dataset
#     def data_generator(shards):
#         for shard in shards:
#             for ex in shard:
#                 yield ex
#     ds = IterableDataset.from_generator(data_generator, gen_kwargs={'shards': domain_ds_ls})
#     logger.info("Not interleaving dataset - will not sample according to domain weights")

# else:
#     ds = interleave_datasets(
#             domain_ds_ls,
#             probabilities=probabilities,
#             probabilities_file=probabilities_tmp_file,
#             seed=seed)
    

# def take_data_generator(ds, max_samples):
#     idx = 0
#     for ex in ds:
#         yield ex
#         idx += 1
#         if max_samples is not None and idx >= max_samples:
#             return

# ds = IterableDataset.from_generator(take_data_generator, gen_kwargs={'ds': ds, 'max_samples': max_samples})
# train_dataset = ds


In [ ]:
# test_ds = load_dataset(
#     "json",
#     data_files='test.jsonl',
#     cache_dir=model_args.cache_dir)['train']
# test_ds[0]

# for x in test_ds.to_iterable_dataset():
#     print(x)
# for i, v in enumerate(ds):
#     if i == 10:
#         break
#     print(v)

In [42]:

if training_args.reweight_domains:
    torch_dtype = (
        model_args.torch_dtype
        if model_args.torch_dtype in ["auto", None]
        else getattr(torch, model_args.torch_dtype)
    )
    if model_args.model_type in {'gpt_flash', 'gpt_neox_flash'}:
        model_cls = doremi_models.GPTFlashAttnLMHeadModel
        reference_model = model_cls.from_pretrained(
            training_args.reference_model_name_or_path,
            config=config)
    else:
        if model_args.model_type in {'gpt2'}:
            model_cls = doremi_models.GPT2LMHeadModelDoReMi
        elif model_args.model_type in {'pythia'}:
            model_cls = doremi_models.GPTNeoXForCausalLMDoReMi
        else:
            model_cls = AutoModelForCausalLM
        reference_model = model_cls.from_pretrained(
            training_args.reference_model_name_or_path,
            from_tf=bool(".ckpt" in model_args.model_name_or_path),
            config=config,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
            torch_dtype=torch_dtype,
        )
    for param in reference_model.parameters():
        param.requires_grad = False
    reference_model.eval()
    model.reference_model = reference_model
    # initialize α0 as α_ref
    model.register_buffer('train_domain_weights', torch.tensor(
            [train_domain_weights_dict[domain] for domain in domain_list]))
    model.register_buffer('avg_domain_weights', model.train_domain_weights.clone())
    model.register_buffer('perdomain_scores', torch.ones(len(train_domain_weights_dict)) * np.log(len(tokenizer)))
    model.register_buffer('update_counter', torch.tensor(1))

else:
    reference_model = None


In [48]:


# turn off find unused parameters
training_args.ddp_find_unused_parameters = False

# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
# embedding_size = model.get_input_embeddings.weight.shape[0]
# if len(tokenizer) > embedding_size:
#     model.resize_token_embeddings(len(tokenizer))

torch.cuda.empty_cache()

# Initialize our Trainer
trainer = DoReMiTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_utils.get_data_collator(
        tokenizer, do_padding=data_args.do_padding, max_seq_length=data_args.max_token_length),
)

trainer

[INFO|trainer.py:565] 2023-08-08 22:39:03,164 >> max_steps is given, it will override any value given in num_train_epochs


In [44]:
print(trainer.args.max_grad_norm)
print(trainer.do_grad_scaling)
print(trainer.args.n_gpu)
print(trainer.args.local_rank)
print(trainer.is_model_parallel)
print(trainer.args.sharded_ddp)
print(trainer.args.fsdp)
print(trainer.is_fsdp_enabled)
print(trainer.args.distributed_state)
print(trainer.args.world_size)

1.0
False
1
0
False
[]
[]
False
Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

1


In [49]:

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    trainer.save_model()  # Saves the tokenizer too for easy upload

    metrics = train_result.metrics

    if training_args.reweight_domains:
        avg_domain_weights_dict = {}
        for i in range(len(model.avg_domain_weights)):
            domain_name = domain_list[i]
            metrics[f'avg_domain_weight:{domain_name}'] = model.avg_domain_weights[i].item()
            avg_domain_weights_dict[domain_name] = model.avg_domain_weights[i].item()

        # save avg domain weights to json
        avg_domain_weights_file = Path(training_args.output_dir) / 'avg_domain_weights.json'
        with open(avg_domain_weights_file, 'w') as f:
            json.dump(avg_domain_weights_dict, f, indent=2)

        # also save to configs dir
        config_dict = {"train_domain_weights": avg_domain_weights_dict,
                       "eval_domain_weights": avg_domain_weights_dict}
        config_dict_file = Path(__file__).parent.parent / 'configs' / f"{Path(training_args.output_dir).name}.json"
        with open(config_dict_file, 'w') as f:
            json.dump(config_dict, f, indent=2)

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

[INFO|trainer.py:1682] 2023-08-08 22:39:04,866 >> ***** Running training *****
[INFO|trainer.py:1683] 2023-08-08 22:39:04,867 >>   Num examples = 199,936
[INFO|trainer.py:1684] 2023-08-08 22:39:04,868 >>   Num Epochs = 9,223,372,036,854,775,807
[INFO|trainer.py:1685] 2023-08-08 22:39:04,869 >>   Instantaneous batch size per device = 1
[INFO|trainer.py:1688] 2023-08-08 22:39:04,870 >>   Total train batch size (w. parallel, distributed & accumulation) = 128
[INFO|trainer.py:1689] 2023-08-08 22:39:04,871 >>   Gradient Accumulation steps = 128
[INFO|trainer.py:1690] 2023-08-08 22:39:04,872 >>   Total optimization steps = 1,562
[INFO|trainer.py:1691] 2023-08-08 22:39:04,874 >>   Number of trainable parameters = 162,322,944


08/08/2023 22:39:04 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2711299ea724aa2d.arrow


[INFO|trainer_utils.py:696] 2023-08-08 22:39:04,922 >> The following columns in the training set don't have a corresponding argument in `GPTNeoXForCausalLMDoReMi.forward` and have been ignored: domain_id. If domain_id are not expected by `GPTNeoXForCausalLMDoReMi.forward`,  you can safely ignore this message.
[WARNING|logging.py:280] 2023-08-08 22:39:04,923 >> You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


08/08/2023 22:39:04 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-5e467f527ede3e03.arrow
08/08/2023 22:39:04 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-0ed22c958e1c1445.arrow
08/08/2023 22:39:05 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-26e80827a48fbfab.arrow



KeyboardInterrupt



In [55]:
data_collator=data_utils.get_data_collator(tokenizer, do_padding=data_args.do_padding)

inputs = list(train_dataset.take(3))
inputs = data_collator(inputs)

outputs = model.forward(**{k:v.to('cuda') for k,v in inputs.items()}, return_pertoken_losses=True)

print(outputs.loss, outputs.pertoken_loss.shape, outputs.token_mask.shape)

08/08/2023 22:40:58 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2711299ea724aa2d.arrow
08/08/2023 22:40:58 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-5e467f527ede3e03.arrow
08/08/2023 22:40:58 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-0ed22c958e1c1445.arrow
tensor(2.3500, device='cuda:0', grad_fn=<DivBackward0>) torch.Size([3, 1023]) torch.Size

In [51]:
data_collator=data_utils.get_data_collator(tokenizer, do_padding=data_args.do_padding)
self = trainer

inputs = list(train_dataset.take(3))
inputs = data_collator(inputs)

model.train()
inputs = self._prepare_inputs(inputs)

with self.compute_loss_context_manager():
    loss, pertoken_loss, reference_pertoken_loss, token_mask = self.compute_loss(model, inputs, return_pertoken_losses=True)
    excess_loss = pertoken_loss - reference_pertoken_loss

loss.shape, pertoken_loss.shape, token_mask.shape, 
                

08/08/2023 22:39:33 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2711299ea724aa2d.arrow
08/08/2023 22:39:33 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-5e467f527ede3e03.arrow
08/08/2023 22:39:33 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-0ed22c958e1c1445.arrow


[WARNING|logging.py:280] 2023-08-08 22:39:33,825 >> You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyError: 'reference_pertoken_loss'

In [ ]:
self = trainer

pertoken_scores = excess_loss
token_masks = token_mask.bool()
domain_ids = inputs['domain_ids']

# self.update_domain_weights(pertoken_scores, token_masks, domain_ids)

scores = pertoken_scores
scores_mask = token_masks


train_domain_weights = self.read_weights()

scores = scores.detach()
domain_ids = domain_ids.detach()


perdomain_scores = []
for domain_id in range(len(train_domain_weights)):
    domain_mask = (domain_ids == domain_id)
    perdomain_scores_mask = scores_mask[domain_mask]
    if domain_mask.sum() > 0:
        curr_domain_scores = torch.clip(scores[domain_mask][perdomain_scores_mask], min=0).mean()
    else:
        curr_domain_scores = self.model.perdomain_scores[domain_id]
    perdomain_scores.append(curr_domain_scores)

# `perdomain_scores` with size (#domains,) is the `λₜ` in the paper, 
#     e.g., the gradient of loss w.r.t. αₜ dL/dαₜ
self.model.perdomain_scores[:] = torch.tensor(perdomain_scores)

# `train_domain_weights` is αₜ; `log_new_train_domain_weights` is log(a_t')
# following implements: α_t' = α_{t-1} exp(η·λ_t), e.g., log(a_t') = log(α_{t-1}) + η·λ_t
log_new_train_domain_weights = torch.log(train_domain_weights+1e-30) + self.args.reweight_eta * self.model.perdomain_scores

# following implements: 
# softmax(log(α_t')) = (exp(log(a_t'[1])) / Σᵢ exp(log(a_t'[i])), ...)
#                    = (a_t'[1]/Σᵢ a_t'[i], ...)
new_train_domain_weights = nn.functional.softmax(log_new_train_domain_weights, dim=-1)

# following implements:
# αₜ = (1-c)·softmax(log(α_t')) + c·u where c=1e-4 by default, and u=1/#domains.
train_domain_weights = (1-self.args.reweight_eps) * new_train_domain_weights + self.args.reweight_eps / len(new_train_domain_weights)

# wpq: clip the weights to [ϵ, 1-ϵ] just to make sure.
eps = 1e-8
train_domain_weights = torch.clip(train_domain_weights, min=eps, max=1-eps)

self.write_weights(train_domain_weights)

In [ ]:
print(pertoken_loss.shape, token_mask.shape)


In [ ]:
# doremiv2
import torch.distributed as dist

# compute the rescaled loss, divide by domain weights
train_domain_weights = self.read_weights().to(pertoken_loss.device)
# if doing non-uniform sampling, normalize by inverse sampling weight
train_domain_weights = train_domain_weights / self.sampling_weights.to(train_domain_weights.device)
train_domain_weights = train_domain_weights / train_domain_weights.sum()

curr_domain_weights = train_domain_weights[inputs['domain_ids']].unsqueeze(-1).expand_as(pertoken_loss).detach()
curr_domain_weights = curr_domain_weights * token_mask

normalizer = curr_domain_weights.sum()

## wpq: remove since `token_mask` not used any more.
# token_mask = token_mask.detach().type(pertoken_loss.dtype)
curr_domain_weights = curr_domain_weights / normalizer # sum to 1.
## wpq: clip `curr_domain_weights` to [0, 1]
curr_domain_weights = torch.clip(curr_domain_weights, min=1e-8, max=1-1e-8)
loss = (pertoken_loss * curr_domain_weights.detach()).sum()
loss



Σ_i α_i Σ_{x\in Dᵢ} |x|

In [ ]:

train_domain_weights = self.read_weights().to(pertoken_loss.device)
# if doing non-uniform sampling, normalize by inverse sampling weight
train_domain_weights = train_domain_weights / self.sampling_weights.to(train_domain_weights.device)
train_domain_weights = train_domain_weights / train_domain_weights.sum()

# (#domains,) total number of tokens amongst samples from each domain
perdomain_num_tokens = []
for domain_id in range(len(train_domain_weights)):
    domain_mask = (inputs['domain_ids'] == domain_id)
    if domain_mask.sum() > 0:
        num_tokens = token_mask[domain_mask].sum()
    else:
        num_tokens = torch.tensor(0., device=token_mask.device)
    perdomain_num_tokens.append(num_tokens)
perdomain_num_tokens = torch.stack(perdomain_num_tokens)

# avoid division by zero
perdomain_num_tokens[torch.where(perdomain_num_tokens==0)] = 1.
# (#domains,) equivalent to αᵢ / Σ_{x\in D_i} |x|
perdomain_coeff = train_domain_weights/perdomain_num_tokens
# (bsz, seq_len-1)
coeff = perdomain_coeff[inputs['domain_ids']].unsqueeze(-1) * token_mask
loss = (pertoken_loss * coeff.detach()).sum()
loss

In [ ]:

# avoid division by zero
perdomain_num_tokens[torch.where(perdomain_num_tokens==0)] = 1.
# (#domains,) equivalent to αᵢ / Σ_{x\in D_i} |x|
perdomain_coeff = train_domain_weights/perdomain_num_tokens
# (bsz, seq_len-1)
coeff = perdomain_coeff[inputs['domain_ids']].unsqueeze(-1) * token_mask
loss = (pertoken_loss * coeff.detach()).sum()

In [ ]:
# compute the rescaled loss, divide by domain weights
train_domain_weights = self.read_weights().to(pertoken_loss.device)
# if doing non-uniform sampling, normalize by inverse sampling weight
train_domain_weights = train_domain_weights / self.sampling_weights.to(train_domain_weights.device)
train_domain_weights = train_domain_weights / train_domain_weights.sum()

curr_domain_weights = train_domain_weights[inputs['domain_ids']].unsqueeze(-1).expand_as(pertoken_loss).detach()
curr_domain_weights = curr_domain_weights * token_mask

normalizer = curr_domain_weights.sum()

## wpq: remove since `token_mask` not used any more.
# token_mask = token_mask.detach().type(pertoken_loss.dtype)
curr_domain_weights = curr_domain_weights / normalizer # sum to 1.
## wpq: clip `curr_domain_weights` to [0, 1]
curr_domain_weights = torch.clip(curr_domain_weights, min=1e-8, max=1-1e-8)
loss = (pertoken_loss * curr_domain_weights.detach()).sum()
loss

In [ ]:

# token_mask = token_mask.detach().type(pertoken_loss.dtype)
# curr_domain_weights = curr_domain_weights / normalizer





In [ ]:

# compute the rescaled loss, divide by domain weights
# train_domain_weights = self.read_weights().to(pertoken_loss.device)
# if doing non-uniform sampling, normalize by inverse sampling weight
train_domain_weights = train_domain_weights / self.sampling_weights.to(train_domain_weights.device)
train_domain_weights = train_domain_weights / train_domain_weights.sum()
train_domain_weights